In [ ]:
# %% [markdown]
# # Home Credit Default Risk - Model Training
# 
# ## 🤖 Overview
# This notebook trains machine learning models to predict loan default risk:
# 1. Load engineered features
# 2. Train multiple models
# 3. Evaluate and compare performance
# 4. Make predictions on test data
# 
# ## 📊 Models to Train
# 1. Logistic Regression (Baseline)
# 2. Random Forest
# 3. XGBoost
# 4. LightGBM (Recommended)
# 5. CatBoost

# %% [markdown]
# ## 📦 Setup and Imports

# %%
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Model imports
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix, roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# Import project modules
import sys
sys.path.append('../src')
from utils import load_config
from model_training import ModelTrainer

# Set display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

# Visualization settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# %%
# Load configuration
config = load_config()
print("Configuration loaded successfully")

# %% [markdown]
# ## 📁 Load Engineered Features

# %%
# Load features
print("Loading engineered features...")

train_path = '../features/train_features.pkl'
test_path = '../features/test_features.pkl'

try:
    train_df = pd.read_pickle(train_path)
    test_df = pd.read_pickle(test_path)
    
    print(f"Training data shape: {train_df.shape}")
    print(f"Test data shape: {test_df.shape}")
    
except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Please run feature engineering notebook first!")
    # If features don't exist, we'll create a simple version
    train_df = None
    test_df = None

# %%
if train_df is not None:
    # Display basic information
    print("\nTraining Data Info:")
    print(f"Samples: {len(train_df)}")
    print(f"Features: {len(train_df.columns) - 2}")  # Excluding SK_ID_CURR and TARGET
    
    if 'TARGET' in train_df.columns:
        target_dist = train_df['TARGET'].value_counts(normalize=True) * 100
        print(f"Target Distribution:")
        print(f"  0 (No Default): {target_dist[0]:.2f}%")
        print(f"  1 (Default): {target_dist[1]:.2f}%")
    
    # Show feature types
    print(f"\nData Types:")
    print(train_df.dtypes.value_counts())

# %% [markdown]
# ## 🎯 Prepare Data for Training

# %%
def prepare_data(df):
    """Prepare data for model training."""
    
    if df is None or 'TARGET' not in df.columns:
        print("Error: Training data not available or TARGET column missing")
        return None
    
    # Separate features and target
    X = df.drop(columns=['TARGET', 'SK_ID_CURR'])
    y = df['TARGET']
    
    # Get feature names
    feature_names = list(X.columns)
    
    print(f"\nData Preparation:")
    print(f"  Features: {len(feature_names)}")
    print(f"  Samples: {len(X)}")
    print(f"  Positive class: {y.sum()} ({100*y.mean():.2f}%)")
    
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, 
        test_size=config['model']['test_size'],
        random_state=config['model']['random_state'],
        stratify=y
    )
    
    print(f"\nTrain/Validation Split:")
    print(f"  Training set: {X_train.shape}")
    print(f"  Validation set: {X_val.shape}")
    print(f"  Train positive: {y_train.sum()} ({100*y_train.mean():.2f}%)")
    print(f"  Val positive: {y_val.sum()} ({100*y_val.mean():.2f}%)")
    
    return X_train, X_val, y_train, y_val, feature_names

# %%
# Prepare data
data = prepare_data(train_df)

if data:
    X_train, X_val, y_train, y_val, feature_names = data

# %% [markdown]
# ## 🤖 Train Logistic Regression (Baseline)

# %%
def train_logistic_regression(X_train, y_train, X_val, y_val):
    """Train and evaluate logistic regression model."""
    
    print("\n" + "="*50)
    print("Training Logistic Regression")
    print("="*50)
    
    # Initialize model
    model = LogisticRegression(
        random_state=config['model']['random_state'],
        class_weight='balanced',
        max_iter=1000,
        solver='liblinear',
        C=0.1  # Regularization strength
    )
    
    # Train model
    print("Training model...")
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Calculate metrics
    auc = roc_auc_score(y_val, y_pred_proba)
    accuracy = accuracy_score(y_val, y_pred)
    
    print(f"\nValidation Results:")
    print(f"  AUC Score: {auc:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")
    
    # Classification report
    print("\nClassification Report:")
    print(classification_report(y_val, y_pred, target_names=['Non-Default', 'Default']))
    
    # Cross-validation
    cv_scores = cross_val_score(
        model, pd.concat([X_train, X_val]), pd.concat([y_train, y_val]),
        cv=5, scoring='roc_auc', n_jobs=-1
    )
    
    print(f"\nCross-Validation (5-fold):")
    print(f"  AUC Scores: {cv_scores}")
    print(f"  Mean AUC: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    
    return model, y_pred_proba, y_pred, auc, accuracy

# %%
# Train logistic regression
if data:
    lr_model, lr_proba, lr_pred, lr_auc, lr_acc = train_logistic_regression(X_train, y_train, X_val, y_val)

# %% [markdown]
# ## 🌲 Train Random Forest

# %%
def train_random_forest(X_train, y_train, X_val, y_val):
    """Train and evaluate random forest model."""
    
    print("\n" + "="*50)
    print("Training Random Forest")
    print("="*50)
    
    # Initialize model
    model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=10,
        min_samples_leaf=5,
        class_weight='balanced',
        random_state=config['model']['random_state'],
        n_jobs=-1
    )
    
    # Train model
    print("Training model...")
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Calculate metrics
    auc = roc_auc_score(y_val, y_pred_proba)
    accuracy = accuracy_score(y_val, y_pred)
    
    print(f"\nValidation Results:")
    print(f"  AUC Score: {auc:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")
    
    # Feature importance
    importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 10 Important Features:")
    for i, row in importance.head(10).iterrows():
        print(f"  {row['feature']}: {row['importance']:.4f}")
    
    return model, y_pred_proba, y_pred, auc, accuracy, importance

# %%
# Train random forest
if data:
    rf_model, rf_proba, rf_pred, rf_auc, rf_acc, rf_importance = train_random_forest(X_train, y_train, X_val, y_val)

# %% [markdown]
# ## ⚡ Train XGBoost

# %%
def train_xgboost(X_train, y_train, X_val, y_val):
    """Train and evaluate XGBoost model."""
    
    print("\n" + "="*50)
    print("Training XGBoost")
    print("="*50)
    
    # Get hyperparameters from config
    params = config['hyperparameters']['xgboost']
    
    # Initialize model
    model = xgb.XGBClassifier(
        n_estimators=params['n_estimators'],
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        random_state=config['model']['random_state'],
        n_jobs=-1,
        eval_metric='auc',
        use_label_encoder=False,
        scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1])  # Handle imbalance
    )
    
    # Train model
    print("Training model...")
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )
    
    # Make predictions
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Calculate metrics
    auc = roc_auc_score(y_val, y_pred_proba)
    accuracy = accuracy_score(y_val, y_pred)
    
    print(f"\nValidation Results:")
    print(f"  AUC Score: {auc:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")
    
    # Feature importance
    importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 10 Important Features:")
    for i, row in importance.head(10).iterrows():
        print(f"  {row['feature']}: {row['importance']:.4f}")
    
    # Plot learning curve
    results = model.evals_result()
    epochs = len(results['validation_0']['auc'])
    x_axis = range(0, epochs)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(x_axis, results['validation_0']['auc'], label='Train')
    ax.legend()
    plt.ylabel('AUC')
    plt.title('XGBoost AUC over Iterations')
    plt.show()
    
    return model, y_pred_proba, y_pred, auc, accuracy, importance

# %%
# Train XGBoost
if data:
    xgb_model, xgb_proba, xgb_pred, xgb_auc, xgb_acc, xgb_importance = train_xgboost(X_train, y_train, X_val, y_val)

# %% [markdown]
# ## 💡 Train LightGBM

# %%
def train_lightgbm(X_train, y_train, X_val, y_val):
    """Train and evaluate LightGBM model."""
    
    print("\n" + "="*50)
    print("Training LightGBM")
    print("="*50)
    
    # Get hyperparameters from config
    params = config['hyperparameters']['lightgbm']
    
    # Initialize model
    model = lgb.LGBMClassifier(
        n_estimators=params['n_estimators'],
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
        num_leaves=params['num_leaves'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        reg_alpha=params['reg_alpha'],
        reg_lambda=params['reg_lambda'],
        random_state=config['model']['random_state'],
        n_jobs=-1,
        class_weight='balanced'
    )
    
    # Train model
    print("Training model...")
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='auc',
        callbacks=[
            lgb.early_stopping(50),
            lgb.log_evaluation(50)
        ]
    )
    
    # Make predictions
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Calculate metrics
    auc = roc_auc_score(y_val, y_pred_proba)
    accuracy = accuracy_score(y_val, y_pred)
    
    print(f"\nValidation Results:")
    print(f"  AUC Score: {auc:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")
    
    # Feature importance
    importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 10 Important Features:")
    for i, row in importance.head(10).iterrows():
        print(f"  {row['feature']}: {row['importance']:.4f}")
    
    # Plot feature importance
    plt.figure(figsize=(12, 8))
    lgb.plot_importance(model, max_num_features=20, importance_type='gain')
    plt.title('LightGBM Feature Importance')
    plt.tight_layout()
    plt.show()
    
    return model, y_pred_proba, y_pred, auc, accuracy, importance

# %%
# Train LightGBM
if data:
    lgb_model, lgb_proba, lgb_pred, lgb_auc, lgb_acc, lgb_importance = train_lightgbm(X_train, y_train, X_val, y_val)

# %% [markdown]
# ## 🐱 Train CatBoost

# %%
def train_catboost(X_train, y_train, X_val, y_val):
    """Train and evaluate CatBoost model."""
    
    print("\n" + "="*50)
    print("Training CatBoost")
    print("="*50)
    
    # Get hyperparameters from config
    params = config['hyperparameters']['catboost']
    
    # Identify categorical columns
    categorical_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Initialize model
    model = cb.CatBoostClassifier(
        iterations=params['iterations'],
        learning_rate=params['learning_rate'],
        depth=params['depth'],
        l2_leaf_reg=params['l2_leaf_reg'],
        random_state=config['model']['random_state'],
        verbose=100,
        cat_features=categorical_features,
        auto_class_weights='Balanced'
    )
    
    # Train model
    print("Training model...")
    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        early_stopping_rounds=50,
        verbose=False
    )
    
    # Make predictions
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    y_pred = (y_pred_proba > 0.5).astype(int)
    
    # Calculate metrics
    auc = roc_auc_score(y_val, y_pred_proba)
    accuracy = accuracy_score(y_val, y_pred)
    
    print(f"\nValidation Results:")
    print(f"  AUC Score: {auc:.4f}")
    print(f"  Accuracy: {accuracy:.4f}")
    
    # Feature importance
    importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 10 Important Features:")
    for i, row in importance.head(10).iterrows():
        print(f"  {row['feature']}: {row['importance']:.4f}")
    
    return model, y_pred_proba, y_pred, auc, accuracy, importance

# %%
# Train CatBoost
if data:
    cb_model, cb_proba, cb_pred, cb_auc, cb_acc, cb_importance = train_catboost(X_train, y_train, X_val, y_val)

# %% [markdown]
# ## 📊 Model Comparison

# %%
def compare_models(model_results):
    """Compare performance of all trained models."""
    
    print("\n" + "="*60)
    print("Model Comparison")
    print("="*60)
    
    # Create comparison DataFrame
    comparison_df = pd.DataFrame(model_results).T
    comparison_df = comparison_df[['auc', 'accuracy']]
    
    print("\nModel Performance Summary:")
    print(comparison_df.sort_values('auc', ascending=False))
    
    # Visual comparison
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. AUC Comparison
    axes[0, 0].barh(range(len(comparison_df)), comparison_df['auc'].sort_values())
    axes[0, 0].set_yticks(range(len(comparison_df)))
    axes[0, 0].set_yticklabels(comparison_df['auc'].sort_values().index)
    axes[0, 0].set_xlabel('AUC Score')
    axes[0, 0].set_title('Model AUC Comparison')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Accuracy Comparison
    axes[0, 1].barh(range(len(comparison_df)), comparison_df['accuracy'].sort_values())
    axes[0, 1].set_yticks(range(len(comparison_df)))
    axes[0, 1].set_yticklabels(comparison_df['accuracy'].sort_values().index)
    axes[0, 1].set_xlabel('Accuracy')
    axes[0, 1].set_title('Model Accuracy Comparison')
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. ROC Curves
    axes[1, 0].set_title('ROC Curves Comparison')
    axes[1, 0].set_xlabel('False Positive Rate')
    axes[1, 0].set_ylabel('True Positive Rate')
    
    for model_name, results in model_results.items():
        if 'proba' in results:
            fpr, tpr, _ = roc_curve(y_val, results['proba'])
            axes[1, 0].plot(fpr, tpr, label=f'{model_name} (AUC = {results["auc"]:.3f})')
    
    axes[1, 0].plot([0, 1], [0, 1], 'k--', label='Random')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. Feature Importance Comparison (for tree-based models)
    tree_models = ['random_forest', 'xgboost', 'lightgbm', 'catboost']
    importance_data = {}
    
    for model_name in tree_models:
        if model_name in model_results and 'importance' in model_results[model_name]:
            importance_data[model_name] = model_results[model_name]['importance'].head(10)
    
    # Create a combined importance plot
    if importance_data:
        all_features = set()
        for imp_df in importance_data.values():
            all_features.update(imp_df['feature'].head(5).tolist())
        
        # Create a DataFrame for the plot
        plot_data = pd.DataFrame(index=list(all_features))
        
        for model_name, imp_df in importance_data.items():
            top_features = imp_df.set_index('feature')['importance'].head(5)
            plot_data[model_name] = top_features
        
        plot_data = plot_data.fillna(0)
        plot_data.plot(kind='bar', ax=axes[1, 1])
        axes[1, 1].set_title('Top Feature Importance Comparison')
        axes[1, 1].set_xlabel('Feature')
        axes[1, 1].set_ylabel('Importance')
        axes[1, 1].legend()
        axes[1, 1].tick_params(axis='x', rotation=45)
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Select best model
    best_model_name = comparison_df['auc'].idxmax()
    best_auc = comparison_df.loc[best_model_name, 'auc']
    
    print(f"\nBest Model: {best_model_name} (AUC: {best_auc:.4f})")
    
    return best_model_name, best_auc

# %%
# Collect model results
if data:
    model_results = {
        'logistic_regression': {
            'auc': lr_auc,
            'accuracy': lr_acc,
            'proba': lr_proba,
            'pred': lr_pred
        },
        'random_forest': {
            'auc': rf_auc,
            'accuracy': rf_acc,
            'proba': rf_proba,
            'pred': rf_pred,
            'importance': rf_importance
        },
        'xgboost': {
            'auc': xgb_auc,
            'accuracy': xgb_acc,
            'proba': xgb_proba,
            'pred': xgb_pred,
            'importance': xgb_importance
        },
        'lightgbm': {
            'auc': lgb_auc,
            'accuracy': lgb_acc,
            'proba': lgb_proba,
            'pred': lgb_pred,
            'importance': lgb_importance
        },
        'catboost': {
            'auc': cb_auc,
            'accuracy': cb_acc,
            'proba': cb_proba,
            'pred': cb_pred,
            'importance': cb_importance
        }
    }
    
    # Compare models
    best_model_name, best_auc = compare_models(model_results)

# %% [markdown]
# ## 💾 Save Best Model

# %%
def save_best_model(model, model_name, importance=None):
    """Save the best model to disk."""
    
    print(f"\nSaving best model: {model_name}")
    
    import pickle
    import os
    
    # Create models directory
    os.makedirs('../models', exist_ok=True)
    
    # Save model
    model_path = f'../models/best_model_{model_name}.pkl'
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"Model saved to: {model_path}")
    
    # Save feature importance if available
    if importance is not None:
        importance_path = f'../models/feature_importance_{model_name}.pkl'
        with open(importance_path, 'wb') as f:
            pickle.dump(importance, f)
        print(f"Feature importance saved to: {importance_path}")
    
    # Also save as best_model.pkl (for easy loading)
    best_model_path = '../models/best_model.pkl'
    with open(best_model_path, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"Best model also saved to: {best_model_path}")
    
    return model_path

# %%
# Save best model
if data:
    # Determine which model object to save based on best_model_name
    model_objects = {
        'logistic_regression': lr_model,
        'random_forest': rf_model,
        'xgboost': xgb_model,
        'lightgbm': lgb_model,
        'catboost': cb_model
    }
    
    importance_objects = {
        'random_forest': rf_importance,
        'xgboost': xgb_importance,
        'lightgbm': lgb_importance,
        'catboost': cb_importance
    }
    
    best_model = model_objects[best_model_name]
    best_importance = importance_objects.get(best_model_name)
    
    saved_path = save_best_model(best_model, best_model_name, best_importance)

# %% [markdown]
# ## 🔮 Make Predictions on Test Data

# %%
def make_predictions(model, test_df, threshold=0.5):
    """Make predictions on test data."""
    
    print("\n" + "="*50)
    print("Making Predictions on Test Data")
    print("="*50)
    
    if test_df is